In [8]:
!pip install -r ../requirements.txt


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:

import biosteam as bst
import pdb
import numpy as np
from scipy.optimize import differential_evolution # population based optimization code
from scipy.stats import qmc # latin hypercube sampling 



In [ ]:

import matplotlib.pyplot as plt
import biosteam as bst
import pdb
import numpy as np
from scipy.optimize import differential_evolution # population based optimization code
from scipy.stats import qmc # latin hypercube sampling 
from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args
import warnings
import traceback
from BayesianNN import *

class BlackBox():
    def __init__(self, verbose=False):
        bst.nbtutorial() # For light-mode diagrams, ignore warnings
        self.set_base_model()
        self.nEval = 0
        self.verbose = verbose
        self.history = []  # 최적화 과정 저장
        print(" ##### An instance of the 'BlackBox' class  has been initialised!")
    
    def set_base_model(self):
        # relevant values based on the website example
        # _n1 = 12 # number of stages for extractor
        # _Lr1, _Hr1, _k1  = [0.95, 0.95, 1.4]
        # _Lr2, _Hr2, _k2  = [0.999, 0.999, 1.4]
        # _T_hex = 310
        # _Lr3, _Hr3, _k3  = [0.99, 0.99, 1.5]
        X = [12, 0.95, 0.95, 0.999, 0.999,  310, 0.99, 0.99]
        self._set(X)
        
    def _bounds(self):
        # bounds for variables. feel free to change!
        bounds = [(0, 50), # no. of stages in extractor
                  (0, 0.9999), (0, 0.9999),  # light key, heavy key, 'k' for extract distiller
                  (0, 0.9999), (0, 0.9999),  # light key, heavy key, 'k' for acetic_acid_purification
                  (273, 350), # temperature for 'HX'
                  (0, 0.9999), (0, 0.9999),  # light key, heavy key, 'k' for reffinate_distiller
                  ]
        return bounds
    
    def _integrality(self):
        # which varibales are integers (True if integer)
        ints = [True, 
                False, False, 
                False, False, 
                False, 
                False, False, 
                ]
        return ints
        
        
    def _set(self, X):
        if len(X) != 8:
            print(f"입력 변수 X의 길이가 올바르지 않습니다:, {len(X)}, {X})")
        # Define chemicals used in the process
        bst.settings.set_thermo(['Water', 'AceticAcid', 'EthylAcetate'])
        
        _n1 = int(X[0])  # 추출기 단계 수 (정수화)
        _Lr1, _Hr1 = X[1], X[2]
        _Lr2, _Hr2 = X[3], X[4]
        _T_hex = X[5]
        _Lr3, _Hr3 = X[6], X[7]
        
        # Amount of ethyl-acetate to fermentation broth
        solvent_feed_ratio = 1.5

        # Fermentation broth with dilute acetic acid
        acetic_acid_broth = bst.Stream(ID='acetic_acid_broth', AceticAcid=1000, Water=9000, units='kg/hr')

        # Solvent
        ethyl_acetate = bst.Stream(ID='ethyl_acetate',  EthylAcetate=1)

        # Products
        glacial_acetic_acid = bst.Stream(ID='glacial_acetic_acid')
        wastewater = bst.Stream(ID='wastewater')

        # Recycles
        solvent_recycle = bst.Stream('solvent_rich')
        water_rich = bst.Stream('water_rich')
        distillate = bst.Stream('raffinate_distillate')
        
        
        # System and unit operations
        with bst.System('AAsep') as sys:
            try:
                print(f"Iteration {self.nEval}: stages: {_n1}")
            except:
                print(f"stages: {_n1}")
            extractor = bst.MultiStageMixerSettlers(
                'extractor',
                ins=(acetic_acid_broth, ethyl_acetate, solvent_recycle),
                outs=('extract', 'raffinate'),
                top_chemical='EthylAcetate',
                feed_stages=(0, -1, -1),
                N_stages=_n1,
                use_cache=True,
            )

            @extractor.add_specification(run=True)
            def adjust_fresh_solvent_flow_rate():
                broth = acetic_acid_broth.F_mass
                EtAc_recycle = solvent_recycle.imass['EthylAcetate']
                EtAc_required = broth * solvent_feed_ratio
                if EtAc_required < EtAc_recycle:
                    solvent_recycle.F_mass *= EtAc_required / EtAc_recycle
                    EtAc_recycle = solvent_recycle.imass['EthylAcetate']
                EtAc_fresh = EtAc_required - EtAc_recycle
                ethyl_acetate.imass['EthylAcetate'] = max(
                    0, EtAc_fresh
                )

            HX = bst.HXutility(
                'extract_heater',
                ins=(extractor.extract),
                outs=('hot_extract'),
                rigorous=True,
                V=0,
            )
            ED = bst.ShortcutColumn(
                'extract_distiller',
                ins=HX-0,
                outs=['', 'acetic_acid'],
                LHK=('Water', 'AceticAcid'),
                Lr=_Lr1,
                Hr=_Hr1,
                k=1.4,
                partial_condenser=False,
            )
            ED2 = bst.ShortcutColumn(
                'acetic_acid_purification',
                ins=ED-1,
                outs=('', glacial_acetic_acid),
                LHK=('EthylAcetate', 'AceticAcid'),
                Lr=_Lr2,
                Hr=_Hr2,
                k=1.4,
                partial_condenser=False
            )
            ED.check_LHK = ED2.check_LHK = False
            mixer = bst.Mixer(
                ins=(ED-0, ED2-0, distillate)
            )
            HX = bst.HXutility(ins=mixer-0, T=_T_hex)
            settler = bst.MixerSettler(
                'settler',
                ins=HX-0,
                outs=(solvent_recycle, water_rich),
                top_chemical='EthylAcetate',
            )
            mixer = bst.Mixer(ins=[extractor.raffinate, water_rich])
            RD = bst.ShortcutColumn(
                'raffinate_distiller',
                LHK=('EthylAcetate', 'Water'),
                ins=mixer-0,
                outs=[distillate, wastewater],
                partial_condenser=False,
                Lr=_Lr3,
                Hr=_Hr3,
                k=1.5,
            )
            
        sys.operating_hours = 330 * 24 # annual operating hours, hr/yr
        
        self.sys = sys

    def capex(self):
        # capex of equipment in MMUSD/yr
        capex = round(self.sys.installed_equipment_cost / 1e6, 4)
        
        try:
            int(capex) # checks if nan or a number is returned
            return capex
        
        except:
            print(f"Iteration {self.nEval}: {capex}")
            print("capex error")
            return 10
        
    def opex(self):
        # opex of equipment in MMUSD/yr
        # opex = round(self.sys.material_cost + self.sys.utility_cost / 1e6, 4)
        opex = round((self.sys.material_cost + self.sys.utility_cost) / 1e6, 4)
        try:
            int(opex) # checks if nan or a number is returned
            return opex
        
        except:
            print(f"Iteration {self.nEval}: {opex}")
            print("opex error")
            return 10
        
    def cost(self):
        return self.capex() + self.opex()
    
    # def revenue(self):
    #     # price taken from website
    #     # https://businessanalytiq.com/procurementanalytics/index/acetic-acid-price-index/
        
    #     if self.acetic_acid_constraint() == 0: 
    #         stream = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid'][0]
    #         P_AceticAcid = 0.4 # $/kg
    #         F_AceticAcid = stream.F_mass
    #         return round(P_AceticAcid * F_AceticAcid * self.sys.operating_hours / 1e6, 4) # Units: $/yr
        
    #     else: # return nothing if not in-specification
    #         print("revenue error")
    #         return 0
    
    # def profit(self):
    #     return self.revenue() - self.cost()
            
    def MSP(self):
        # stream = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid'][0]
        streams = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid']
        if not streams:
            print(f"Iteration {self.nEval}: glacial_acetic_acid 스트림이 생성되지 않았습니다.")
            return np.nan
        stream = streams[0]
        P_AceticAcid = 0.4 # $/kg
        F_AceticAcid = stream.F_mass * self.sys.operating_hours / 1e6 # kg/yr

        constraint_violation = self.acetic_acid_constraint()  # 제약 위반 정도 (0이면 만족, 클수록 심각한 위반)
    
        if constraint_violation != 0: # return profit if in-specification
            # 제약 위반 → 패널티 적용 (순도 부족 정도에 따라 패널티 크기 조정)
            print(f"Iteration {self.nEval}: MSP penalty: constraint_violation")
            msp = self.cost() / F_AceticAcid # UNits: $/kg
            penalty_factor = constraint_violation  # (예: 순도 부족 1% -> 10배 패널티, 5% 부족 -> 50배 패널티)
            return round(msp + 100 * penalty_factor, 4)
        else:
            msp = self.cost() / F_AceticAcid # UNits: $/kg
            return round(msp, 4) 
        
    def simulate(self):
        # self.nEval += 1
        self.sys.simulate()

    def wt_acetic_acid(self):
        stream = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid'][0]
        streams = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid']
        if not streams:
            print(f"Iteration {self.nEval}: glacial_acetic_acid 스트림이 생성되지 않았습니다.")
            return np.nan
        stream = streams[0]
        return stream.get_mass_fraction(IDs='AceticAcid')
        
    def acetic_acid_constraint(self):
        x_desired = 0.98 # wt%
        x_achieved = self.wt_acetic_acid()
        d_x = x_desired - x_achieved # -ve if  product is in-spec
        
        cons = max(0, d_x) # return 0 if happy or constraint violation if not
        print(f"Iteration {self.nEval}: Purity {x_achieved}, Constraint: {cons}")
        return cons
        
    def func(self, X=None):
        self.nEval += 1  # 실행 횟수 증가
        try:
            if X is None:
                self.set_base_model()
                
            else:# set and run the simulation
                self._set(X) # set the new operating parameters
                    
            self.simulate() # run the simulation
            
            # assess plant financials
            objective_function = self.MSP()
            
        # if failure for any reason, then reutrn a value of np.inf
        # except:
        #     print('func error')
        #     return 50
        except Exception as e:
            print(f"Iteration {self.nEval}: func error: {e}")
            traceback.print_exc()  # 전체 스택 트레이스 출력
            return 50


        if self.verbose:
            print(self.nEval, objective_function)
            
        return objective_function
    
    def natural_units(self, X):
        X_natural = np.zeros((np.shape(X)))
        bounds = self._bounds()
        b = np.array(bounds)
        d_b = b[:,1] - b[:,0] # range of bounds in natural units
        
        for i, x in enumerate(X):
            X_natural[i] = b[:,0] + (x[:] * d_b)
        
        return X_natural
            
    def optimize(self, method='DE', maxiter=50):
        
        # simple differential evolution approach based on scipy implementation 
        if method == 'DE':
            bounds = self._bounds()
            sampler = qmc.LatinHypercube(d=len(bounds))
            sample = sampler.random(n=100) # n is the population size!
            print("Normalized initial samples:")
            print(sample)
            print("Initial population in natural units:")
            print(population)
            
            population = self.natural_units(sample)
            def callback(xk, convergence):
                """
                실행 횟수 (nEval)와 MSP 값 저장
                """
                
                msp_value = self.func(xk)  # 현재 MSP 계산
                self.history.append([self.nEval, msp_value, self.wt_acetic_acid()])  # 실행 횟수와 MSP 저장

                if self.verbose:
                    print(f"Iteration {self.nEval}: MSP = {msp_value}")
            result = differential_evolution(
                self.func,
                bounds=self._bounds(),
                integrality=self._integrality(),
                init=population,
                maxiter=maxiter,  # 최대 반복 횟수 설정
                callback=callback,  # 실행 결과 저장
            )

            return result
        elif method == 'BO':  # Bayesian Optimization (Gaussian Process)
            print("Running Bayesian Optimization (BO)")
    
            # Search Space 정의 (정수형 & 실수형 혼합)
            space = [
                Integer(0, 50, name="n_stages"),  # Extractor의 단계 수
                Real(0, 0.9999, name="Lr1"),
                Real(0, 0.9999, name="Hr1"),
                Real(0, 0.9999, name="Lr2"),
                Real(0, 0.9999, name="Hr2"),
                Real(273, 350, name="T_hex"),
                Real(0, 0.9999, name="Lr3"),
                Real(0, 0.9999, name="Hr3"),
            ]
            def objective(x):
                """
                Bayesian Optimization을 위한 목적 함수
                x: 리스트 형태의 입력 변수
                """
                msp_value = self.func(x)
                purity = self.wt_acetic_acid()
                
                self.history.append([self.nEval, msp_value, purity])  # 실행 횟수와 MSP 저장
            
                if self.verbose:
                    print(f"Iteration {self.nEval}: MSP = {msp_value}")
            
                return msp_value  # MSP 값을 최소화하는 방향으로 최적화
    
            # Bayesian Optimization 실행
            result = gp_minimize(
                objective,  # 목적 함수
                dimensions=space,  # 탐색 공간
                n_calls=maxiter,  # 최대 반복 횟수
                random_state=42,  # 재현 가능성을 위한 난수 고정
                n_initial_points=10,  # 초기 샘플링 개수 (탐색 안정성 확보)
                acq_func="EI",  # Expected Improvement (EI) 사용
            )
            return result
        elif method == 'BNN':
            return self.active_learning_bnn(maxiter=maxiter, initial_samples=10)
    
    def active_learning_bnn(self, maxiter=20, initial_samples=10):
        """
        Bayesian Neural Network 기반 Active Learning을 사용하여 최적의 설계 변수 찾기
        """
        print("Running Bayesian Neural Network Active Learning")
    
        bounds = np.array(self._bounds())
        input_dim = len(bounds)
    
        #initial data sampling (Latin Hypercube Sampling)
        sampler = qmc.LatinHypercube(d=input_dim)
        sample = sampler.random(n=initial_samples)
        X_train = self.natural_units(sample)
        y_train = np.array([self.func(x) for x in X_train]).reshape(-1, 1)
    
        # PyTorch Tensor transformation
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    
        #inintialise model
        model = BayesianNN(input_dim)
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.MSELoss()
    
        #initial training
        for epoch in range(100):
            optimizer.zero_grad()
            outputs = model(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()
    
        #Active Learning 
        for i in range(maxiter):
            # 4.1 generate new sample (Latin Hypercube Sampling)
            new_samples = sampler.random(n=5)
            X_candidates = self.natural_units(new_samples)
    
            # 4.2 Using MC Dropoutto evaluate uncertainty
            model.train()  # Dropout 활성화
            X_candidates_tensor = torch.tensor(X_candidates, dtype=torch.float32)
    
            predictions = torch.stack([model(X_candidates_tensor) for _ in range(50)])  # 50번 샘플링
            uncertainties = predictions.std(dim=0).detach().numpy().flatten()
    
            # NaN 발생 여부 체크
            if np.isnan(uncertainties).any():
                print("Uncertainty contains NaN values!")
                
            # 4.3 select most uncertain data
            idx_max_uncertainty = np.argmax(uncertainties)
            X_new = X_candidates[idx_max_uncertainty]
    
            # 4.4 evaluate new data
            y_new = self.func(X_new)
            X_train = np.vstack([X_train, X_new])
            y_train = np.vstack([y_train, [y_new]])
    
            # PyTorch Tensor update
            X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
            y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    
            # 4.5 retrain the model!
            for epoch in range(50):
                optimizer.zero_grad()
                outputs = model(X_train_tensor)
                loss = criterion(outputs, y_train_tensor)
                loss.backward()
                optimizer.step()
    
            # 최적화 진행 과정 저장
            # self.history.append([len(self.history) + 1, y_new])
            self.history.append([self.nEval, y_new, self.wt_acetic_acid()])
    
            if self.verbose:
                print(f"Iteration {i + 1}: Selected Sample MSP = {y_new:.4f}, Uncertainty = {uncertainties[idx_max_uncertainty]:.4f}")
    
        return X_train, y_train
    
    def plot_results(self):
        """
        최적화 실행 횟수별 MSP 값 그래프 출력
        """
        if not self.history:
            print("No optimization history found. Run optimize() first.")
            return

        history_array = np.array(self.history)  # shape: (N, 3)
        iterations = history_array[:, 0]        # nEval
        msps = history_array[:, 1]             # MSP
        x_achieved_arr = history_array[:, 2]   # x_achieved

        fig, ax1 = plt.subplots(figsize=(8, 5))
        
        # 첫 번째 Y축: MSP
        color1 = 'tab:blue'
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('MSP ($/kg)', color=color1)
        ax1.plot(iterations, msps, marker='o', color=color1, label='MSP ($/kg)')
        ax1.tick_params(axis='y', labelcolor=color1)
        ax1.grid(True, which='both', linestyle='--', alpha=0.3)
        
        # 두 번째 Y축: x_achieved
        ax2 = ax1.twinx()
        color2 = 'tab:red'
        ax2.set_ylabel('wt_fraction of AceticAcid', color=color2)
        ax2.plot(iterations, x_achieved_arr, marker='x', color=color2, label='AceticAcid purity')
        ax2.tick_params(axis='y', labelcolor=color2)

        plt.title("Optimization progress: MSP & AceticAcid purity vs. Iteration")
        fig.tight_layout()
        plt.show()
        





In [ ]:

#pdb.set_trace() # left this here incase you need to debug something
# bb = BlackBox(verbose=True)
# results = bb.optimize(maxiter=100) 
# bb.plot_results()

bb = BlackBox(verbose=True)
results_bo = bb.optimize(method='DE', maxiter=10)  # Bayesian Optimization 
bb.plot_results()


stages: 12
 ##### An instance of the 'BlackBox' class  has been initialised!
Iteration 1: stages: 0
Iteration 1: func error: list index out of range
Iteration 2: stages: 1


Traceback (most recent call last):
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 265, in func
    self._set(X) # set the new operating parameters
    ^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 86, in _set
    with bst.System('AAsep') as sys:
         ^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_system.py", line 1234, in __exit__
    if exception: raise exception
                  ^^^^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 91, in _set
    extractor = bst.MultiStageMixerSettlers(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/units/stage.py", line 1205, in __init__
    Unit.__init__(self, ID, ins, outs, thermo, **kwargs)
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_unit.py", li

Iteration 2: Purity 0.014875014192117574, Constraint: 0.9651249858078824
Iteration 2: MSP penalty: constraint_violation
2 96.6382
Iteration 3: stages: 1
Iteration 3: Purity 0.011006588973083776, Constraint: 0.9689934110269162
Iteration 3: MSP penalty: constraint_violation
3 96.9614
Iteration 4: stages: 1
Iteration 4: Purity 0.8240944832399145, Constraint: 0.15590551676008546
Iteration 4: MSP penalty: constraint_violation
4 16.4032
Iteration 5: stages: 0
Iteration 5: func error: list index out of range
Iteration 6: stages: 1
Iteration 6: Purity 0.017149043271684004, Constraint: 0.962850956728316
Iteration 6: MSP penalty: constraint_violation
6 96.3588
Iteration 7: stages: 1


Traceback (most recent call last):
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 265, in func
    self._set(X) # set the new operating parameters
    ^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 86, in _set
    with bst.System('AAsep') as sys:
         ^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_system.py", line 1234, in __exit__
    if exception: raise exception
                  ^^^^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 91, in _set
    extractor = bst.MultiStageMixerSettlers(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/units/stage.py", line 1205, in __init__
    Unit.__init__(self, ID, ins, outs, thermo, **kwargs)
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_unit.py", li

Iteration 7: Purity 0.07614461298521331, Constraint: 0.9038553870147866
Iteration 7: MSP penalty: constraint_violation
7 90.7638
Iteration 8: stages: 1
Iteration 8: Purity 0.2802469222458269, Constraint: 0.6997530777541731
Iteration 8: MSP penalty: constraint_violation
8 70.5159
Iteration 9: stages: 0
Iteration 9: func error: list index out of range
Iteration 10: stages: 0
Iteration 10: func error: list index out of range
Iteration 11: stages: 1


...

Traceback (most recent call last):
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 267, in func
    self.simulate() # run the simulation
    ^^^^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 238, in simulate
    self.sys.simulate()
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_system.py", line 2793, in simulate
    with self.flowsheet:
         ^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_flowsheet.py", line 120, in __exit__
    if exception: raise exception
                  ^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_system.py", line 2849, in simulate
    raise error
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_system.py", line 2837, in simulate
    if design_and_cost: self._summary()
                        ^^^^^^^^^^^^^^^
  File "/Users/k23070952/

Iteration 76: Purity 0.08125227998939566, Constraint: 0.8987477200106043
Iteration 76: MSP penalty: constraint_violation
76 90.1611
Iteration 77: stages: 0
Iteration 77: func error: list index out of range
Iteration 78: stages: 0
Iteration 78: func error: list index out of range
Iteration 79: stages: 1
Iteration 79: Purity 0.09606756882847986, Constraint: 0.8839324311715201
Iteration 79: MSP penalty: constraint_violation
79 88.5108
Iteration 80: stages: 0
Iteration 80: func error: list index out of range
Iteration 81: stages: 1


Traceback (most recent call last):
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 265, in func
    self._set(X) # set the new operating parameters
    ^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 86, in _set
    with bst.System('AAsep') as sys:
         ^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_system.py", line 1234, in __exit__
    if exception: raise exception
                  ^^^^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 91, in _set
    extractor = bst.MultiStageMixerSettlers(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/units/stage.py", line 1205, in __init__
    Unit.__init__(self, ID, ins, outs, thermo, **kwargs)
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_unit.py", li

Iteration 81: Purity 0.014853942972949615, Constraint: 0.9651460570270504
Iteration 81: MSP penalty: constraint_violation
Iteration 81: nan
capex error
81 97.6427
Iteration 82: stages: 0
Iteration 82: func error: list index out of range
Iteration 83: stages: 1
Iteration 83: Purity 0.04600507005443265, Constraint: 0.9339949299455673
Iteration 83: MSP penalty: constraint_violation
83 93.4639
Iteration 84: stages: 1
Iteration 84: Purity 0.002123857148480912, Constraint: 0.977876142851519
Iteration 84: MSP penalty: constraint_violation
Iteration 84: nan
capex error
84 101.7133
Iteration 85: stages: 1
Iteration 85: Purity 0.06306390110328546, Constraint: 0.9169360988967146
Iteration 85: MSP penalty: constraint_violation
85 91.7701
Iteration 86: stages: 1


Traceback (most recent call last):
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 265, in func
    self._set(X) # set the new operating parameters
    ^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 86, in _set
    with bst.System('AAsep') as sys:
         ^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_system.py", line 1234, in __exit__
    if exception: raise exception
                  ^^^^^^^^^^^^^^^
  File "/var/folders/c9/14_8p05x241fyxw1d_hqydvr0000gq/T/ipykernel_32183/213112036.py", line 91, in _set
    extractor = bst.MultiStageMixerSettlers(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/units/stage.py", line 1205, in __init__
    Unit.__init__(self, ID, ins, outs, thermo, **kwargs)
  File "/Users/k23070952/.local/lib/python3.12/site-packages/biosteam/_unit.py", li

In [1]:
bb.plot_results()

NameError: name 'bb' is not defined